In [10]:
import os
from icrawler.builtin import GoogleImageCrawler
from ultralytics import YOLO
from PIL import Image
from tqdm import tqdm

COCO_FRUITS = {"apple", "banana", "orange"}

fruit_keywords = {
    "Apple": ["apple fruit", "red apple", "green apple", "apple on tree"],
    "Banana": ["banana fruit", "ripe banana", "banana bunch", "yellow banana"],
    "Mango": ["mango fruit", "ripe mango", "yellow mango", "mango on tree"],
    "Orange": ["orange fruit", "ripe orange", "orange slice", "orange citrus"],
    "Grapes": ["grapes fruit", "green grapes", "purple grapes", "grapes bunch"],
    "Pomegranate": ["pomegranate fruit", "ripe pomegranate", "pomegranate seeds"],
    "Watermelon": ["watermelon fruit", "sliced watermelon", "ripe watermelon"],
    "Guava": ["guava fruit", "green guava", "ripe guava"],
    "Papaya": ["papaya fruit", "ripe papaya", "papaya slice"],
    "Strawberry": ["strawberry fruit", "fresh strawberry", "ripe strawberries"]
}

def download_images(fruit, queries, num_images_per_query=400):
    save_dir = f'raw_images/{fruit}'
    os.makedirs(save_dir, exist_ok=True)

    for i, query in enumerate(queries):
        print(f" {fruit}: Downloading from query '{query}'...")
        crawler = GoogleImageCrawler(storage={'root_dir': f'{save_dir}/part_{i}'})
        crawler.crawl(keyword=query, max_num=num_images_per_query)

def merge_and_flatten(fruit):
    base_dir = f'raw_images/{fruit}'
    all_parts = [os.path.join(base_dir, part) for part in os.listdir(base_dir)]
    flat_dir = f'merged_images/{fruit}'
    os.makedirs(flat_dir, exist_ok=True)

    count = 0
    for part in all_parts:
        for fname in os.listdir(part):
            src = os.path.join(part, fname)
            dst = os.path.join(flat_dir, f"{fruit}_{count}.jpg")
            try:
                img = Image.open(src)
                img.save(dst)
                count += 1
            except:
                continue
    return flat_dir

def filter_images_with_yolo(fruit, input_dir, conf_threshold=0.3):
    yolo_model = YOLO("yolov8n.pt")
    output_dir = f'cleaned_dataset/{fruit}'
    os.makedirs(output_dir, exist_ok=True)

    for fname in tqdm(os.listdir(input_dir), desc=f"🔍 Filtering {fruit}"):
        img_path = os.path.join(input_dir, fname)
        try:
            img = Image.open(img_path)
            #aspect ratio checking
            if img.size[0] < 150 or img.size[1] < 150 or img.size[0]/img.size[1] > 3 or img.size[1]/img.size[0] > 3:
                continue

            results = yolo_model(img_path, conf=conf_threshold)[0]
            detected_labels = [results.names[int(cls)] for cls in results.boxes.cls]

            if fruit.lower() in COCO_FRUITS:
                if any(label in COCO_FRUITS for label in detected_labels):
                    img.save(os.path.join(output_dir, fname))
            else:
                if len(detected_labels) > 0:
                    img.save(os.path.join(output_dir, fname))

        except Exception:
            continue

def run_full_pipeline():
    for fruit, queries in fruit_keywords.items():
        print(f"\n===PROCESSING: {fruit} ===")
        download_images(fruit, queries, num_images_per_query=400)
        merged_path = merge_and_flatten(fruit)
        filter_images_with_yolo(fruit, merged_path)

    print("\nDataset ready in 'cleaned_dataset/' folder.")

if __name__ == "__main__":
    run_full_pipeline()


2025-07-10 00:43:36,919 - INFO - icrawler.crawler - start crawling...
2025-07-10 00:43:36,920 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-07-10 00:43:36,922 - INFO - feeder - thread feeder-001 exit
2025-07-10 00:43:36,926 - INFO - icrawler.crawler - starting 1 parser threads...
2025-07-10 00:43:36,929 - INFO - icrawler.crawler - starting 1 downloader threads...



===PROCESSING: Apple ===
 Apple: Downloading from query 'apple fruit'...


2025-07-10 00:43:38,231 - INFO - parser - parsing result page https://www.google.com/search?q=apple+fruit&ijn=0&start=0&tbs=&tbm=isch
2025-07-10 00:43:39,348 - INFO - downloader - image #1	https://static.libertyprim.com/files/familles/pomme-large.jpg
2025-07-10 00:43:39,627 - INFO - downloader - image #2	https://cdn.britannica.com/22/187222-050-07B17FB6/apples-on-a-tree-branch.jpg
2025-07-10 00:43:40,103 - INFO - downloader - image #3	https://healthjade.com/wp-content/uploads/2017/10/apple-fruit.jpg
2025-07-10 00:43:40,967 - INFO - downloader - image #4	https://www.realfruitpower.com/RealFruit/RealFruitImages/457/image-thumb__457__full-banner/contentimage7-8-2014873623971.42b35659.png
2025-07-10 00:43:41,459 - INFO - downloader - image #5	https://www.nirvanaorganic.in/cdn/shop/articles/matheus-cenali-wXuzS9xR49M-unsplash.jpg
2025-07-10 00:43:41,769 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Pink_lady_and_cross_section

 Apple: Downloading from query 'red apple'...


2025-07-10 00:44:53,774 - INFO - parser - parsing result page https://www.google.com/search?q=red+apple&ijn=0&start=0&tbs=&tbm=isch
2025-07-10 00:44:55,396 - INFO - downloader - image #1	https://botanix.com/cdn/shop/files/Malus-domestica-Red-Gala_fe2a1904-c46a-49f6-aa4d-cc4cd8ce111c.jpg
2025-07-10 00:44:56,025 - INFO - downloader - image #2	https://static.vecteezy.com/system/resources/previews/005/170/799/non_2x/red-apple-with-apple-slices-and-leaves-vitamins-healthy-food-fruit-on-a-white-background-realistic-3d-illustration-vector.jpg
2025-07-10 00:44:58,711 - ERROR - downloader - Exception caught when downloading file https://png, error: HTTPSConnectionPool(host='png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000285108B6470>: Failed to resolve 'png' ([Errno 11001] getaddrinfo failed)")), remaining retry times: 2
2025-07-10 00:45:01,409 - ERROR - downloader - Exception caught when downloading file htt

...


image 1/1 C:\Users\Reyyan Saeed\merged_images\Strawberry\Strawberry_98.jpg: 480x640 4 apples, 78.7ms
Speed: 4.3ms preprocess, 78.7ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)


🔍 Filtering Strawberry:  99%|██████████████████████████████████████████████████████▋| 169/170 [00:21<00:00,  8.06it/s]


image 1/1 C:\Users\Reyyan Saeed\merged_images\Strawberry\Strawberry_99.jpg: 640x480 3 pizzas, 73.7ms
Speed: 3.9ms preprocess, 73.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)


🔍 Filtering Strawberry: 100%|███████████████████████████████████████████████████████| 170/170 [00:21<00:00,  7.74it/s]


Dataset ready in 'cleaned_dataset/' folder.
